## Example 3: Building Conversational Chains (Messages and Tools)

Building sophisticated agents that can handle conversations and using tools. 

Reference: [Getting Started with LangGraph: A Beginner’s Guide to Building Intelligent Workflows](https://medium.com/@ashutoshsharmaengg/getting-started-with-langgraph-a-beginners-guide-to-building-intelligent-workflows-67eeee0899d0)

### Messages
The messages captures the different roles in the conversation. This is supported by LangChain. 

- `HumanMessage`: What the user says
- `AIMessage`: What the AI says
- `SystemMessage`: Instructions for the AI/LLM model (or system prompt)
- `ToolMessage`: Output of a tool / function invokation

In [5]:
from dotenv import load_dotenv
import os

# This loads the .env file for loading api-key
load_dotenv() 

True

In [9]:
from pprint import pprint
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# "messages" variable initialized as a list
# 'messages' is list of HumanMessage/AIMessage
messages = [AIMessage(content=f'Are you studying about LangGraph?',name='LLM')]                # LLM's question to the user

# extend() --> adds elements to the messages list from another list
messages.extend([HumanMessage(content=f'Yes', name='Aditi')])                                       # Human's response to LLM 
messages.extend([AIMessage(content=f'What would you want to learn?',name='LLM')])              # LLM's follow-up question to user 
messages.extend([HumanMessage(content=f'I want to learn about stategraph', name='Aditi')])          # Human's response to model

for msg in messages:
    msg.pretty_print()

================================== Ai Message ==================================
Name: LLM

Are you studying about LangGraph?
================================ Human Message =================================
Name: Aditi

Yes
================================== Ai Message ==================================
Name: LLM

What would you want to learn?
================================ Human Message =================================
Name: Aditi

I want to learn about stategraph


We will take the list of these messages and will pass it to our chat model.

In [16]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')
result = llm.invoke(messages) 
print(result.content)

StateGraph is a term that can refer to various concepts depending on the context, including computer science, graph theory, and software engineering. In general, it can involve the representation of states and transitions between those states in a graphical format. 

Here are a few common uses of StateGraph:

1. **Finite State Machines (FSMs)**: A StateGraph can represent an FSM, where nodes represent states and edges represent transitions based on inputs or events. This is commonly used in designing protocols, game development, and control systems.

2. **State Management in Programming**: In the context of applications, especially single-page applications (SPAs), a StateGraph may represent the different states of the application and how it transitions from one state to another, often used in frameworks like React or Vue.js.

3. **Model Checking**: In formal verification, StateGraphs are used to represent the possible states of a system and their transitions, allowing for the analysis 

### Tools (Tool calling by LLMs)
When AI Agent needs to interact with an external function / API / model / websearch / access database.

- **Tool Creation**: `@tool` decorator to create a tool
    - A tool is an association between a Python function and its schema (which describes its purpose and required inputs)
- **Tool Binding**:
    - The tool needs to be connected to a LLM model that supports tool calling.
    - Gives the model awareness of the tool and the associated input schema required by the tool
- **Tool Calling**: The model can decide to “call” a tool
    - It ensures its response conforms to the tool’s input schema, providing the necessary arguments
- **Tool Execution**: Tool can then be executed using the arguments provided by the model
    - Output is returned to the model to inform its next response

In [21]:
from langchain_core.tools import tool

# Defining a "multiply" tool
@tool
def multiply(a:int, b:int) ->int:
    "Multiply a and b"
    return a*b

# Binding tool with LLM model
llm_with_tool = llm.bind_tools([multiply])

# Tool call 
tool_call = llm_with_tool.invoke('what is 2 multiply by 3')
# There is no content or "result" in AIMessage, but there is a tool call
# print(tool_call)

# Shows the tool call structure
print(tool_call.additional_kwargs['tool_calls']) 

[{'id': 'call_sXqm9EhwrfkuzaP8dk63GTmM', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}]


### Using Messages as State & Reducers
In a conversational agent, this is to ensure that each node will not override the message history. Tells LangChain to append new messages as a list.

The `add_messages` reducer ensures that when a node returns a `messages` key, its content is appended to the existing list, rather than replacing it.

In [23]:
from typing import TypedDict, Annotated # adds metadata
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages # Reducer

# Tells LangGraph to append messages as list
class MessageState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages] 

### A Graph with MessageState and tool calling
- It uses `MessageState` to manage conversation history
- It has an LLM node that can decide to call tools
- It has a `ToolNode` that executes any tools the LLM requests
- It uses a conditional edge (`tools_condition`) to route to the `ToolNode` if the LLM calls a tool.

In [25]:
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI

In [26]:
# Define some tools 
@tool
def multiply(a:int, b:int)->int:
    """this tool will do multiplication"""
    return a*b

@tool
def add(a:int, b:int) -> int:
    "Add two integers"
    return a+b

@tool
def subtract(a:int, b:int) -> int:
    "Subtract two integers"
    return a-b
    
tools_list = [multiply, add, subtract]

# Bind tools to your LLM
llm_with_tool = ChatOpenAI(model='gpt-4o-mini').bind_tools(tools_list)

`ToolNode`: LangGraph pre-built class that wraps external tools or functions
- Triggered when a tool call is detected
- Enables the agent to execute the appropriate function and return the result to the workflow

In [27]:
# Define the state for our conversational agent
class State(TypedDict):
    messages : Annotated[list[AnyMessage], add_messages]

# Define a node for the LLM chatbot
def llm_chatbot(state: State):
    # Invoke the LLM with the current message history
    return {'messages': [llm_with_tool.invoke(state['messages'])]}

# ToolNode --> defines the tools as a node
# Will run the tools requested by the last AIMessage
# If there are multiple tools called, it will run in parallel
tool_node = ToolNode(tools_list) # Accepts a list of tools

`tools_condition`: pre-built LangGraph function that checks if the last state message includes a tool call
- If it does, the flow is routed to the tools node for execution

In [28]:
# Build the StateGraph
build = StateGraph(State)
# LLM chatbot node
build.add_node('LLM', llm_chatbot)
# Tool node to execute tools
build.add_node('tools', tool_node) 

# Starting point
build.add_edge(START, 'LLM') # Start by sending user input to the LLM

# Add a conditional edge from 'LLM'
build.add_conditional_edges(
    "LLM",
    tools_condition, # This is a pre-built LangGraph condition: if last message has tool calls, it routes to "tools"
    # The default mapping for tools_condition is {"tools": "tools_node_name"}
)

# After tools run, send results back to the LLM for next turn
build.add_edge('tools', 'LLM') 

# Compile for results
app = build.compile()

#### Example Invocations:
1. Natural response (LLM directly answers)

In [34]:
# print(app.invoke({'messages': 'Hi'}))
res = app.invoke({'messages': 'Hi'})
for r in res['messages']:
    r.pretty_print()

================================ Human Message =================================

Hi
================================== Ai Message ==================================

Hello! How can I assist you today?


2. Tool response (LLM calls tools, then provides an answer)

In [32]:
msg = app.invoke({'messages': "first tell me some things about Boston. Second calculate 10+2"})
for m in msg['messages']:
    m.pretty_print()

================================ Human Message =================================

first tell me some things about Boston. Second calculate 10+2
================================== Ai Message ==================================

Boston is the capital city of Massachusetts and is one of the oldest cities in the United States, founded in 1630. Here are some key points about Boston:

1. **History**: Boston played a significant role in American history, especially during the American Revolution, with events such as the Boston Massacre and the Boston Tea Party.

2. **Education**: The city is known for its prestigious universities and colleges, including Harvard University and the Massachusetts Institute of Technology (MIT), both located in the Greater Boston area.

3. **Culture**: Boston has a rich cultural scene, with numerous museums, theaters, and historical sites. The Boston Symphony Orchestra and Boston Museum of Fine Arts are notable institutions.

4. **Sports**: Boston is home to severa

**NOTE**: Expected output will show AI messages, tool call messages, and then the final AI response, combining the search result and the calculation.